Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

## TODOS
1. DONE Debugging, does output make sense?
    1. Resize images
    2. preprocessing fixes
    5. replace scaling by proper function
2. try on leaderboard
3. Include Odometry and fuse into heads
    - Speed
    - Location
4. navigation
5. controller



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy

import os
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline')
import os
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing
from torch.utils.data import DataLoader

In [38]:
class MyResnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torchvision.models.resnet18(weights=True)
        num_ftrs = self.net.fc.in_features
        """
        self.net.fc = nn.Sequential(
            nn.Linear(512, 3), #nbr of outputs self.net.fc.in_features
            nn.Tanh()
        )
        """
        self.net.fc = nn.Identity()
        
        self.thr_head = nn.Sequential(
            nn.Linear(num_ftrs, 50),
            nn.Tanh(),
            nn.Linear(50, 1),
            nn.Sigmoid() # TODO BU
        )
        
        self.brk_head = nn.Sequential(
            nn.Linear(num_ftrs, 50),
            nn.Tanh(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )
        
        self.str_head = nn.Sequential(
            nn.Linear(num_ftrs, 50),
            nn.Tanh(),
            nn.Linear(50, 1),
            nn.Tanh()
        )
        """
        self.net = self.net.cuda() if device else self.net
        self.net.fc = self.net.fc.cuda() if device else self.net.fc
        
        self.thr_head = self.thr_head.cuda() if device else self.thr_head
        self.brk_head = self.brk_head.cuda() if device else self.brk_head
        self.str_head = self.str_head.cuda() if device else self.str_head
        """

    def forward(self, data_):
        x = self.net(data_)
        x = self.net.fc(x)
        return self.thr_head(x), self.str_head(x), self.brk_head(x)

In [47]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet()
net = net.cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

Data Loaders etc


In [48]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Town10HD_Scenario10_route9_11_28_17_59_39 has varying number of data files among input folders. It got discarded from the dataset.


In [49]:
"""
def normalize_img(tensor):
    v_min, v_max = tensor.min(), tensor.max()
    new_min, new_max = 0, 1
    v_p = (tensor - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    return v_p
"""

'\ndef normalize_img(tensor):\n    v_min, v_max = tensor.min(), tensor.max()\n    new_min, new_max = 0, 1\n    v_p = (tensor - v_min)/(v_max - v_min)*(new_max - new_min) + new_min\n    return v_p\n'

In [50]:
def normalize_batch(tensors):
    liste = []
    for tensor in tensors:
        tensor = preprocess(tensor ) #* 1/255
        liste.append(tensor)
    return torch.stack(liste)     

In [51]:
from PIL import Image
from torchvision import transforms
#input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

img = next(iter(train_dataloader))["rgb"].transpose(1,4)[:,:,:,:,0].float()
img.shape

torch.Size([64, 3, 160, 960])

In [52]:
normalize_batch(img).shape#[0,:,:,:]

torch.Size([64, 3, 160, 960])

Training

In [53]:
criterion = nn.L1Loss()#nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)#optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [54]:
%%time

n_epochs = 5
#print_every = 10
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader):

        # further preprocessing
        data_ = normalize_batch(data["rgb"].transpose(1,4)[:,:,:,:,0]).float() # HOTFIX
        #data["rgb"].transpose(1,4)[:,:,:,:,0]
        data["throttle"]=data["throttle"].float()
        data["steer"]=data["steer"].float()
        data["brake"]=data["brake"].float()
        
        #print(target_.shape)
        #print(target_.shape[0])
        """
        if data["throttle"].shape[0] != batch_size: # HOTFIX
            run = False
            break
        """

        # move to GPU
        data_ = data_.to(device)
        
        data["throttle"] = data["throttle"].to(device)
        data["steer"] = data["steer"].to(device)
        data["brake"] = data["brake"].to(device)
        
        # compute outputs
        optimizer.zero_grad()
        
        outputs = net(data_)
        output_throttle = outputs[0].to(device)
        output_steer = outputs[1].to(device)
        output_brake = outputs[2].to(device)
        
        loss_throttle = criterion(output_throttle, data["throttle"].float())
        loss_steer = criterion(output_steer, data["steer"].float())
        loss_brake = criterion(output_brake, data["brake"].float())
        
        
        loss = sum([loss_throttle, loss_steer, loss_brake])/3
        
        # Backprop
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    """
    if not run: # HOTFIX
        continue
    """
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0

    
    """
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    """

    net.train()
    

Epoch 1

Epoch [1/5], Step [0/32], Loss: 0.3565

train-loss: 0.2811,
Epoch 2

Epoch [2/5], Step [0/32], Loss: 0.1683

train-loss: 0.2176,
Epoch 3

Epoch [3/5], Step [0/32], Loss: 0.1119

train-loss: 0.1876,
Epoch 4

Epoch [4/5], Step [0/32], Loss: 0.1607

train-loss: 0.1704,
Epoch 5

Epoch [5/5], Step [0/32], Loss: 0.0882

train-loss: 0.1585,
Wall time: 5min 32s


Test predictions

In [55]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(next(iter(test_dataloader)).keys())


dict_keys(['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])


In [56]:
idx, X = next(enumerate(test_dataloader))

In [57]:
data_ = normalize_batch(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = (X["throttle"],X["steer"],X["brake"])

In [58]:
with torch.no_grad():
    outputs_ = net(data_)

In [59]:
print(outputs_[0])

tensor([[0.0070],
        [0.1184],
        [0.6570],
        [0.6668],
        [0.0070],
        [0.6541],
        [0.6750],
        [0.5240],
        [0.5926],
        [0.6023],
        [0.6734],
        [0.6724],
        [0.6352],
        [0.0072],
        [0.6737],
        [0.6748],
        [0.6696],
        [0.6708],
        [0.6707],
        [0.6326],
        [0.6653],
        [0.0070],
        [0.6729],
        [0.0591],
        [0.4483],
        [0.6686],
        [0.5509],
        [0.0071],
        [0.6688],
        [0.0071],
        [0.6639],
        [0.0084],
        [0.6711],
        [0.6575],
        [0.0074],
        [0.0070],
        [0.6737],
        [0.0077],
        [0.6663],
        [0.0072],
        [0.0078],
        [0.6567],
        [0.6712],
        [0.0121],
        [0.0071],
        [0.6749],
        [0.6362],
        [0.6602],
        [0.0073],
        [0.6715],
        [0.6643],
        [0.6461],
        [0.1179],
        [0.6380],
        [0.6764],
        [0

In [60]:
print(target_[0])

tensor([[0.0000],
        [0.7500],
        [0.7500],
        [0.7500],
        [0.0000],
        [0.2800],
        [0.7500],
        [0.0603],
        [0.0734],
        [0.0461],
        [0.7500],
        [0.7500],
        [0.7500],
        [0.0000],
        [0.0222],
        [0.7500],
        [0.7500],
        [0.7500],
        [0.0547],
        [0.3725],
        [0.0000],
        [0.0000],
        [0.7500],
        [0.0000],
        [0.7500],
        [0.3926],
        [0.7500],
        [0.0000],
        [0.7500],
        [0.0000],
        [0.3043],
        [0.0000],
        [0.7500],
        [0.7500],
        [0.0000],
        [0.0000],
        [0.3934],
        [0.0000],
        [0.6199],
        [0.0000],
        [0.7500],
        [0.7500],
        [0.3571],
        [0.0000],
        [0.0000],
        [0.7500],
        [0.7500],
        [0.0501],
        [0.0000],
        [0.7500],
        [0.0000],
        [0.4280],
        [0.0497],
        [0.7500],
        [0.7500],
        [0

save & load

In [61]:
#torch.save(net, 'rgb_resnet.pth')

In [18]:
#net = torch.load('rgb_resnet.pth')

Time test

In [19]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 37 sekunden
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
at = time.time()
for batch_idx, data in enumerate(train_dataloader):
    data_ = normalize_batch(data["rgb"].transpose(1,4)[:,:,:,:,0]).float()
et = time.time()
print(et-at)


29.58383321762085
